In [1]:
import pandas as pd
import requests, json
from datetime import datetime, timedelta

# Specify the API endpoint URL and API key
API_ENDPOINT = 'https://api.stormglass.io/v2/weather/point'
API_KEY = '29df6d46-d2e5-11ed-bce5-0242ac130002-29df6df0-d2e5-11ed-bce5-0242ac130002'

# LATITUDE, LONGITUDE를 각각 입력하지 않더라도 원하는 지점의 데이터를 받아버린다.
# Specify the location and time range for the data you want to retrieve
locations = [{"lat": 46.633, "lng": 175.140}, {"lat": 46.75, "lng": -139.391}]
start_time = datetime(2023, 1, 1)
end_time = start_time + timedelta(days = 30)
# vessels과 관련하여 필요하다고 생각되는 파라미터
# waterTemperature: 물의 온도는 찰 수록 저항이 세진다
# windSpeed: 배의 속도와 방향에 영향
# waveHeight: 파고
# wavePeriod: 진동수? -> 선박의 안전에 영향을 준다. 짧을 수록 위험
# currentSpeed: ocean current -> 물의 흐름도 측정
# 이 때 좀 이쁘게 정리된 거 처럼 보이고 싶다해서 ,사이에 띄어쓰면 바로 실패
PARAMETERS = 'waterTemperature,waveHeight,windSpeed,wavePeriod,currentSpeed'

weather_data = []


# API 요청 보내고 데이터 추출
while end_time < datetime.now():
    # response 변수를 미리 선언
    for loc in locations:
        response = requests.get(API_ENDPOINT, params={'lat': loc["lat"],
                                                      "lng": loc["lng"],
                                                     'start': start_time.isoformat()+'+00:00', 
                                                     'end': end_time.isoformat()+'+00:00', 
                                                     'params': PARAMETERS},
                                headers={'Authorization': '29df6d46-d2e5-11ed-bce5-0242ac130002-29df6df0-d2e5-11ed-bce5-0242ac130002'},
                               )
    # status_code가 200이면 정상적으로 api를 받아온다.
    if response.status_code == 200:
        weather_data.append(response.json()["hours"])
    else:
        print(f"Request failed {response.status_code}")
        break
    # 시간 갱신
    start_time = end_time
    end_time = start_time + timedelta(days = 30)

Request failed 402


meto, sg, icon, meteo, noaa 등 key값은 데이터 제공업체이다

In [3]:
weather_data

[[{'currentSpeed': {'meto': 0.12, 'sg': 0.12},
   'time': '2023-01-01T00:00:00+00:00',
   'waterTemperature': {'meto': 10.14, 'noaa': 9.71, 'sg': 10.14},
   'waveHeight': {'icon': 3.85, 'meteo': 5.35, 'noaa': 3.72, 'sg': 5.35},
   'wavePeriod': {'icon': 10.19, 'meteo': 8.24, 'noaa': 13.96, 'sg': 8.24},
   'windSpeed': {'icon': 12.54, 'noaa': 13.67, 'sg': 12.54}},
  {'currentSpeed': {'meto': 0.11, 'sg': 0.11},
   'time': '2023-01-01T01:00:00+00:00',
   'waterTemperature': {'meto': 10.14, 'noaa': 9.7, 'sg': 10.14},
   'waveHeight': {'icon': 3.99, 'meteo': 5.34, 'noaa': 3.93, 'sg': 5.34},
   'wavePeriod': {'icon': 10.13, 'meteo': 8.24, 'noaa': 13.83, 'sg': 8.24},
   'windSpeed': {'icon': 13.16, 'noaa': 14.39, 'sg': 13.16}},
  {'currentSpeed': {'meto': 0.09, 'sg': 0.09},
   'time': '2023-01-01T02:00:00+00:00',
   'waterTemperature': {'meto': 10.13, 'noaa': 9.7, 'sg': 10.13},
   'waveHeight': {'icon': 4.12, 'meteo': 5.32, 'noaa': 4.14, 'sg': 5.32},
   'wavePeriod': {'icon': 10.08, 'meteo': 

In [42]:
weather_df = pd.DataFrame(weather_data[0])
for i in range(1, len(weather_data)):
    d = pd.DataFrame(weather_data[i])
    weather_df = pd.concat([weather_df, d], axis = 0, ignore_index = True)

In [43]:
weather_df = weather_df.reindex(columns = ["time", "currentSpeed", "waterTemperature", "waveHeight", "wavePeriod", "windSpeed"])

In [44]:
# 데이터 제공업체 별로 단위 주의
weather_df

,time,currentSpeed,waterTemperature,waveHeight,wavePeriod,windSpeed
0,2023-01-01T00:00:00+00:00,"{'meto': 0.12, 'sg': 0.12}","{'meto': 10.14, 'noaa': 9.71, 'sg': 10.14}","{'icon': 3.85, 'meteo': 5.35, 'noaa': 3.72, 's...","{'icon': 10.19, 'meteo': 8.24, 'noaa': 13.96, ...","{'icon': 12.54, 'noaa': 13.67, 'sg': 12.54}"
1,2023-01-01T01:00:00+00:00,"{'meto': 0.11, 'sg': 0.11}","{'meto': 10.14, 'noaa': 9.7, 'sg': 10.14}","{'icon': 3.99, 'meteo': 5.34, 'noaa': 3.93, 's...","{'icon': 10.13, 'meteo': 8.24, 'noaa': 13.83, ...","{'icon': 13.16, 'noaa': 14.39, 'sg': 13.16}"
2,2023-01-01T02:00:00+00:00,"{'meto': 0.09, 'sg': 0.09}","{'meto': 10.13, 'noaa': 9.7, 'sg': 10.13}","{'icon': 4.12, 'meteo': 5.32, 'noaa': 4.14, 's...","{'icon': 10.08, 'meteo': 8.24, 'noaa': 13.71, ...","{'icon': 13.78, 'noaa': 15.12, 'sg': 13.78}"
3,2023-01-01T03:00:00+00:00,"{'meto': 0.07, 'sg': 0.07}","{'meto': 10.13, 'noaa': 9.69, 'sg': 10.13}","{'icon': 4.26, 'meteo': 5.31, 'noaa': 4.35, 's...","{'icon': 10.02, 'meteo': 8.24, 'noaa': 13.58, ...","{'icon': 14.4, 'noaa': 15.84, 'sg': 14.4}"
4,2023-01-01T04:00:00+00:00,"{'meto': 0.04, 'sg': 0.04}","{'meto': 10.13, 'noaa': 9.71, 'sg': 10.13}","{'icon': 4.44, 'meteo': 5.29, 'noaa': 4.59, 's...","{'icon': 9.94, 'meteo': 8.24, 'noaa': 13.5, 's...","{'icon': 15.03, 'noaa': 17.25, 'sg': 15.03}"
...,...,...,...,...,...,...
718,2023-03-11T20:00:00+00:00,"{'meto': 0.16, 'sg': 0.16}","{'meto': 9.09, 'noaa': 7.93, 'sg': 9.09}","{'icon': 1.86, 'meteo': 1.87, 'noaa': 1.72, 's...","{'icon': 8.2, 'meteo': 5.39, 'noaa': 7.82, 'sg...","{'icon': 6.34, 'noaa': 5.33, 'sg': 6.34}"
719,2023-03-11T21:00:00+00:00,"{'meto': 0.16, 'sg': 0.16}","{'meto': 9.09, 'noaa': 7.93, 'sg': 9.09}","{'icon': 1.85, 'meteo': 1.87, 'noaa': 1.71, 's...","{'icon': 8.29, 'meteo': 5.39, 'noaa': 7.79, 's...","{'icon': 6.21, 'noaa': 5.48, 'sg': 6.21}"
720,2023-03-11T22:00:00+00:00,"{'meto': 0.16, 'sg': 0.16}","{'meto': 9.1, 'noaa': 7.93, 'sg': 9.1}","{'icon': 1.84, 'meteo': 1.86, 'noaa': 1.73, 's...","{'icon': 8.34, 'meteo': 5.42, 'noaa': 7.75, 's...","{'icon': 6.28, 'noaa': 5.71, 'sg': 6.28}"
721,2023-03-11T23:00:00+00:00,"{'meto': 0.15, 'sg': 0.15}","{'meto': 9.1, 'noaa': 7.93, 'sg': 9.1}","{'icon': 1.84, 'meteo': 1.86, 'noaa': 1.74, 's...","{'icon': 8.39, 'meteo': 5.46, 'noaa': 7.71, 's...","{'icon': 6.35, 'noaa': 5.93, 'sg': 6.35}"


In [47]:
# weather_df.to_csv("weather_data_stormglass")
import pandas as pd
pd.read_csv("weather_data_stormglass")

,time,currentSpeed,waterTemperature,waveHeight,wavePeriod,windSpeed
0,2023-01-01T00:00:00+00:00,"{'meto': 0.12, 'sg': 0.12}","{'meto': 10.14, 'noaa': 9.71, 'sg': 10.14}","{'icon': 3.85, 'meteo': 5.35, 'noaa': 3.72, 's...","{'icon': 10.19, 'meteo': 8.24, 'noaa': 13.96, ...","{'icon': 12.54, 'noaa': 13.67, 'sg': 12.54}"
1,2023-01-01T01:00:00+00:00,"{'meto': 0.11, 'sg': 0.11}","{'meto': 10.14, 'noaa': 9.7, 'sg': 10.14}","{'icon': 3.99, 'meteo': 5.34, 'noaa': 3.93, 's...","{'icon': 10.13, 'meteo': 8.24, 'noaa': 13.83, ...","{'icon': 13.16, 'noaa': 14.39, 'sg': 13.16}"
2,2023-01-01T02:00:00+00:00,"{'meto': 0.09, 'sg': 0.09}","{'meto': 10.13, 'noaa': 9.7, 'sg': 10.13}","{'icon': 4.12, 'meteo': 5.32, 'noaa': 4.14, 's...","{'icon': 10.08, 'meteo': 8.24, 'noaa': 13.71, ...","{'icon': 13.78, 'noaa': 15.12, 'sg': 13.78}"
3,2023-01-01T03:00:00+00:00,"{'meto': 0.07, 'sg': 0.07}","{'meto': 10.13, 'noaa': 9.69, 'sg': 10.13}","{'icon': 4.26, 'meteo': 5.31, 'noaa': 4.35, 's...","{'icon': 10.02, 'meteo': 8.24, 'noaa': 13.58, ...","{'icon': 14.4, 'noaa': 15.84, 'sg': 14.4}"
4,2023-01-01T04:00:00+00:00,"{'meto': 0.04, 'sg': 0.04}","{'meto': 10.13, 'noaa': 9.71, 'sg': 10.13}","{'icon': 4.44, 'meteo': 5.29, 'noaa': 4.59, 's...","{'icon': 9.94, 'meteo': 8.24, 'noaa': 13.5, 's...","{'icon': 15.03, 'noaa': 17.25, 'sg': 15.03}"
...,...,...,...,...,...,...
718,2023-03-11T20:00:00+00:00,"{'meto': 0.16, 'sg': 0.16}","{'meto': 9.09, 'noaa': 7.93, 'sg': 9.09}","{'icon': 1.86, 'meteo': 1.87, 'noaa': 1.72, 's...","{'icon': 8.2, 'meteo': 5.39, 'noaa': 7.82, 'sg...","{'icon': 6.34, 'noaa': 5.33, 'sg': 6.34}"
719,2023-03-11T21:00:00+00:00,"{'meto': 0.16, 'sg': 0.16}","{'meto': 9.09, 'noaa': 7.93, 'sg': 9.09}","{'icon': 1.85, 'meteo': 1.87, 'noaa': 1.71, 's...","{'icon': 8.29, 'meteo': 5.39, 'noaa': 7.79, 's...","{'icon': 6.21, 'noaa': 5.48, 'sg': 6.21}"
720,2023-03-11T22:00:00+00:00,"{'meto': 0.16, 'sg': 0.16}","{'meto': 9.1, 'noaa': 7.93, 'sg': 9.1}","{'icon': 1.84, 'meteo': 1.86, 'noaa': 1.73, 's...","{'icon': 8.34, 'meteo': 5.42, 'noaa': 7.75, 's...","{'icon': 6.28, 'noaa': 5.71, 'sg': 6.28}"
721,2023-03-11T23:00:00+00:00,"{'meto': 0.15, 'sg': 0.15}","{'meto': 9.1, 'noaa': 7.93, 'sg': 9.1}","{'icon': 1.84, 'meteo': 1.86, 'noaa': 1.74, 's...","{'icon': 8.39, 'meteo': 5.46, 'noaa': 7.71, 's...","{'icon': 6.35, 'noaa': 5.93, 'sg': 6.35}"
